Please check the Live Notebook from here : https://www.kaggle.com/code/saurabhmaulekhi/parallel-dot-assignments/edit/run/255438169

# Installing sam2

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything-2.gitzzz

  Cloning https://github.com/facebookresearch/segment-anything-2.gitzzz to /tmp/pip-req-build-w9rtf0_e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2.gitzzz /tmp/pip-req-build-w9rtf0_e
Username for 'https://github.com': 

# Imporing Libraries

In [ ]:
import torch
import numpy as np
import pandas as pd
import cv2
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor

from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import os,glob,shutil
import matplotlib.patches as patches

# Data Preprocessing

In [ ]:
objects_list = ["can_chowder","can_soymilk", "can_tomatosoup", "carton_oj", "carton_soymilk", 
                "diet_coke", "hc_potroastsoup", "juicebox", "rice_tuscan", "ricepilaf"]

In [ ]:
len(objects_list)

In [ ]:
numbers = [str(i).zfill(6) for i in range(1, 51)]

In [ ]:
initial_dir = "/kaggle/input/cmu10-3d/CMU10_3D/data_2D/"

In [ ]:
can_chowder_images_list = [initial_dir + objects_list[0] + '_' + num + '.jpg' for num in numbers  ]
can_soymilk_images_list = [initial_dir + objects_list[1] + '_' + num + '.jpg' for num in numbers  ]
can_tomatosoup_images_list = [initial_dir + objects_list[2] + '_' + num + '.jpg' for num in numbers  ]
carton_oj_images_list = [initial_dir + objects_list[3] + '_' + num + '.jpg' for num in numbers  ]
carton_soymilk_images_list = [initial_dir + objects_list[4] + '_' + num + '.jpg' for num in numbers  ]
diet_coke_images_list = [initial_dir + objects_list[5] + '_' + num + '.jpg' for num in numbers  ]
hc_potroastsoup_images_list = [initial_dir + objects_list[6] + '_' + num + '.jpg' for num in numbers  ]
juicebox_images_list = [initial_dir + objects_list[7] + '_' + num + '.jpg' for num in numbers  ]
rice_tuscan_images_list = [initial_dir + objects_list[8] + '_' + num + '.jpg' for num in numbers  ]
ricepilaf_images_list = [initial_dir + objects_list[9] + '_' + num + '.jpg' for num in numbers  ]

In [ ]:
# Nested list containg each and every image files

Objects_images_list = [can_chowder_images_list,
                        can_soymilk_images_list,
                        can_tomatosoup_images_list,
                        carton_oj_images_list,
                        carton_soymilk_images_list,
                        diet_coke_images_list,
                        hc_potroastsoup_images_list,
                        juicebox_images_list,
                        rice_tuscan_images_list,
                        ricepilaf_images_list]

In [ ]:
# Filtering the mask files.
# Appending masked images from a image to a single list 

files_list = os.listdir(initial_dir)

mask_file_list = []

for i in files_list:
    if "gt" in i:
        mask_file_list.append(initial_dir + i)

In [ ]:
from collections import defaultdict

# Group by object name and main number
grouped = defaultdict(lambda: defaultdict(list))

for f in mask_file_list:
    parts = f.split("_")
    obj_name = "_".join(parts[:-3])       # everything before the main number
    main_num = parts[-3]                  # the 6-digit number
    grouped[obj_name][main_num].append(f)

# Convert to nested, nested list
result = [
    [grouped[obj][num] for num in sorted(grouped[obj])]
    for obj in sorted(grouped)
]

In [ ]:
can_chowder_mask_list = result[0]
can_soymilk_mask_list = result[1]
can_tomatosoup_mask_list = result[2]
carton_oj_mask_list = result[3]
carton_soymilk_mask_list = result[4]
diet_coke_mask_list = result[5]
hc_potroastsoup_mask_list = result[6]
juicebox_mask_list = result[7]
rice_tuscan_mask_list = result[8]
ricepilaf_mask_list = result[9]

In [ ]:
# List containg all the mask images path

Objects_mask_list = [can_chowder_mask_list,
                        can_soymilk_mask_list,
                        can_tomatosoup_mask_list,
                        carton_oj_mask_list,
                        carton_soymilk_mask_list,
                        diet_coke_mask_list,
                        hc_potroastsoup_mask_list,
                        juicebox_mask_list,
                        rice_tuscan_mask_list,
                        ricepilaf_mask_list]

In [ ]:
# Load SAM2 model
model_cfg = "sam2_hiera_b+.yaml"
checkpoint = "/kaggle/input/segment-anything-2/pytorch/sam2-hiera-base-plus/1/sam2_hiera_base_plus.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"

predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device=device))

In [ ]:
def find_BBox(obj_image, mask_image):
    """
    obj_image: image file path
    mask_image : mask image file path

    return : BBox of given mask , [xmin, xmax, ymin, ymax ]
    """
    image = cv2.imread(obj_image)
    predictor.set_image(image)
    
    # Load mask
    mask = cv2.imread(mask_image, cv2.IMREAD_GRAYSCALE)
    ys, xs = np.where(mask > 0)

    # If no object pixels found → return zeros
    if xs.size == 0 or ys.size == 0:
        return [0, 0, 0, 0]
    
    xmin, xmax = xs.min(), xs.max()
    ymin, ymax = ys.min(), ys.max()

    return [xmin, xmax, ymin, ymax]

In [ ]:
# Bounding Boxes of each object from an initial image and mask

BBox_can_chowder =[(find_BBox(can_chowder_images_list[0], can_chowder_mask_list[0][0]), 0 )]

BBox_can_soymilk = [(find_BBox(can_soymilk_images_list[0], can_soymilk_mask_list[1][0]), 1 )]

BBox_can_tomatosoup = [(find_BBox(can_tomatosoup_images_list[0], can_tomatosoup_mask_list[2][0]), 2 )]

BBox_carton_oj = [(find_BBox(carton_oj_images_list[0], carton_oj_mask_list[3][0]), 3 )]

BBox_carton_soymilk = [(find_BBox(carton_soymilk_images_list[0], carton_soymilk_mask_list[4][0]), 4 )]

BBox_diet_coke = [(find_BBox(diet_coke_images_list[0], diet_coke_mask_list[5][0]), 5 )]

BBox_hc_potroastsoup = [(find_BBox(hc_potroastsoup_images_list[0], hc_potroastsoup_mask_list[6][0]), 6 )]

BBox_juicebox = [(find_BBox(juicebox_images_list[0], juicebox_mask_list[7][0]), 7 )]

BBox_rice_tuscan = [(find_BBox(rice_tuscan_images_list[32], rice_tuscan_mask_list[9][0]), 8 )]

BBox_ricepilaf = [(find_BBox(ricepilaf_images_list[11], ricepilaf_mask_list[9][0]), 9 )]

In [ ]:
# list of BBox of initial image and objects

Objests_BBox_list = [ BBox_can_chowder,
                    BBox_can_soymilk,
                    BBox_can_tomatosoup,
                    BBox_carton_oj,
                    BBox_carton_soymilk,
                    BBox_diet_coke,
                    BBox_hc_potroastsoup,
                    BBox_juicebox,
                    BBox_rice_tuscan,
                    BBox_ricepilaf,
                    ]

# Segementation

In [ ]:
checkpoint = "/kaggle/input/sam2_hiera_tiny.pt/pytorch/default/1/sam2_hiera_tiny.pt"
model_cfg = "sam2_hiera_t.yaml"

# checkpoint = "/kaggle/input/segment-anything-2.1/pytorch/sam2.1-hiera-base-plus/1/sam2.1_hiera_base_plus.pt"
# model_cfg = "/kaggle/input/segment-anything-2.1/pytorch/sam2.1-hiera-base-plus/1sam2.1_hiera_b.yaml"

predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))
sam2 = build_sam2(model_cfg, checkpoint, device='cuda', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint,device='cuda')

In [ ]:
tempfolder = "/kaggle/working/tempdir"
mask_dir = "/kaggle/working/mask_dir"

In [ ]:
def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

In [ ]:
create_if_not_exists(mask_dir)

In [ ]:
def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

In [ ]:
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size)

In [ ]:
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0,)))

In [ ]:
def track_item_boxes(imgpath1,imgpath2,img1boxclasslist,visualize=True):
    # imgpath1 :: Image where object is known
    # imgpath2 :: Image where object is to be tracked
    # img1boxclasslist :: [ ([xmin,xmax,ymin,ymax],objectnumint) ,....] for all objects in im

    create_if_not_exists(tempfolder)
    cleardir(tempfolder)

    shutil.copy(imgpath1,tempfolder+"/00000.jpg")
    shutil.copy(imgpath2,tempfolder+"/00001.jpg")
    
    #inference_state stores context for tracking across frames.
    inference_state = predictor_vid.init_state(video_path="/kaggle/working/tempdir")

    #Resets any existing tracker memory to a clean state before adding annotations.
    predictor_vid.reset_state(inference_state)

    # Index of the frame that contains the known object location.
    # Here 0 means the first frame (00000.jpg).
    ann_frame_idx = 0

    # Loops through all (bounding box, object_id) pairs in the list.
    # Unpacks the box coordinates into variables xmin, xmax, ymin, ymax.
    # objectnumint is the integer ID for that object.
    for img1boxclass in img1boxclasslist:
    
        ([xmin,xmax,ymin,ymax],objectnumint) = img1boxclass
        box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
        _, out_obj_ids, out_mask_logits = predictor_vid.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=objectnumint,
        box=box,
        )
        
    video_segments = {} # video_segments contains the per-frame segmentation results. ex: {frame_index: {object_id: binary_mask_array}}
        
    # Calls propagate_in_video(inference_state) which:
    # Runs the tracker over all frames. Yields (frame_index, object_ids, mask_logits) for each frame.
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):

        # For each frame:
        # Creates a dictionary mapping each object_id → binary mask array.
        # (out_mask_logits[i] > 0.0) turns the mask logits into a boolean mask (True = object present).
        # .cpu().numpy() moves it from GPU to CPU and converts to a NumPy array
        video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        
            for i, out_obj_id in enumerate(out_obj_ids)
        }
    
    return video_segments

In [ ]:
# listy containg index for each object

Objects_idx = [ x for x in range(10)]

In [ ]:
## to hide the output complition bars by SAM2
import sys, os, builtins
orig_out, orig_err, orig_print = sys.stdout, sys.stderr, print
sys.stdout = sys.stderr = open(os.devnull, 'w')
builtins.print = lambda *a, **k: orig_print(*a, file=orig_out, **k)
#=====================================================#

for object_images_list , object_BBox, Object_idx in zip(Objects_images_list, Objests_BBox_list, Objects_idx):

    print('['+objects_list[Object_idx]+ ']', end="")
    
    for object_num in range(50): # Because each type of object has 50 images
        
        result = track_item_boxes(object_images_list[0],     # image segemntation
                                  object_images_list[object_num],
                                  object_BBox
                                 )
        
        mask = result[1][Object_idx]  # from our SAM2 output
        mask = mask.squeeze()  # remove extra dims
        mask = (mask.astype(np.uint8)) * 255  # scale to 0-255 , it's binary 0/1
        

        new_masked_img_path = mask_dir + "/"+objects_list[Object_idx] + '_' + numbers[object_num]+ "_1_gt.png"
        
        cv2.imwrite(new_masked_img_path, mask) # saving new segmented mask image
        
        print("-", end="")

    print(">","✅")


sys.stdout, sys.stderr, builtins.print = orig_out, orig_err, orig_print # Reseting outputs, so that other cells could show complition bar 

# Loss Calculation

In [ ]:
# Flat list are non nested list that contain just the images path 

Flat_Objects_images_list = [item for sublist in Objects_images_list for item in sublist ]

Flat_Objects_mask_list = [inner[0] for outer in Objects_mask_list for inner in outer]

Predicted_mask_list = [mask_dir+"/"+ x for x in os.listdir(mask_dir)]

In [ ]:
# sorting list

Objects_images_list.sort()
Objects_mask_list.sort()
Predicted_mask_list.sort()

In [ ]:
# Calculating the BBox from both truth anfd predicted segmentations and storing them in a list

from tqdm import tqdm

Truth_BBox = [ find_BBox(img, mask) for img, mask in tqdm(zip(Flat_Objects_images_list, Flat_Objects_mask_list))] 

Predicted_BBox = [ find_BBox(img, mask) for img, mask in tqdm(zip(Flat_Objects_images_list, Predicted_mask_list))]

In [ ]:
print(Truth_BBox[0])
print(Predicted_BBox[0])

In [ ]:
def bbox_iou_Error(truthBBox, predictedBBox):
    """
    truthBBox : BBox of from true mask
    predictedBBox : BBox from predicted mask

    return : Error of Inter section over union between truthBBox and predictedBBox 
    """
    
    # Unpack
    txmin, txmax, tymin, tymax = truthBBox
    pxmin, pxmax, pymin, pymax = predictedBBox

    # Intersection coordinates
    ixmin = max(txmin, pxmin)
    iymin = max(tymin, pymin)
    ixmax = min(txmax, pxmax)
    iymax = min(tymax, pymax)

    # Intersection area
    inter_width = max(0, ixmax - ixmin)
    inter_height = max(0, iymax - iymin)
    inter_area = inter_width * inter_height

    # Areas of truth and predicted boxes
    truth_area = (txmax - txmin) * (tymax - tymin)
    pred_area = (pxmax - pxmin) * (pymax - pymin)

    # Union area
    union_area = truth_area + pred_area - inter_area
    
    # IoU
    return (1 - (inter_area / union_area if union_area > 0 else 0.0))

print(bbox_iou_Error([371, 464, 150, 290],[374, 463, 154, 290]))

In [ ]:
# List of Error/loss between corresponding masks

BBox_Errors_list = [bbox_iou_Error(truthBBox, predictedBBox) for  truthBBox, predictedBBox in zip(Truth_BBox, Predicted_BBox)] 

In [ ]:
# Getting list of images

Objects_image_names = [objects_list[x]+ '_' + num + ".png"  for x in range(10) for num in numbers ]

# Submission file

In [ ]:
import pandas  as pd


df = pd.DataFrame({
    'Image Name': Objects_image_names,
    'Error Between the Orignal and predicted Bounding Box': BBox_Errors_list
})


In [ ]:
df.to_csv("parallel_dots_assignment_submission.csv", index=False)

In [ ]:
df.head()